# LSTM Single Model

## Data Description

    - Raw data: Historical Product Demand.csv

    - Input data: Data on 8x augmentation of demand records by selecting 8 representative items

    - Product code: 'Product_0025', 'Product_0739', 'Product_0901', 'Product_1154',
                    'Product_1248', 'Product_1295', 'Product_1378', 'Product_2004'
            

    - Size of Data: 116392 rows × 4 columns

    - Features: Date, Product_Code, Product_Category, Order_Demand

    - Period: 2012-01-01 ~ 2017-01-09

---

In [5]:
# DataFrame
import pandas as pd
import numpy as np
import random
from datetime import datetime, date

# Preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Save the log
import os
import time
import pickle 

# LSTM
import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import RepeatVector, TimeDistributed, Bidirectional

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Metric 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score

## Data Explore

In [6]:
# Data Loading
df = pd.read_csv('Data\HPD_0416.csv')
# convert the string to the datetype
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
start_date = pd.to_datetime('2012-01-10')
end_date = pd.to_datetime('2016-12-21')

df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
df = df.reset_index(drop=True)
df

,Date,Product_Code,Product_Category,Order_Demand
0,2012-01-10,Product_0025,Category_005,600.0
1,2012-01-11,Product_0025,Category_005,800.0
2,2012-01-12,Product_0025,Category_005,600.0
3,2012-01-13,Product_0025,Category_005,1500.0
4,2012-01-14,Product_0025,Category_005,0.0
...,...,...,...,...
14459,2016-12-17,Product_2004,Category_005,0.0
14460,2016-12-18,Product_2004,Category_005,0.0
14461,2016-12-19,Product_2004,Category_005,0.0
14462,2016-12-20,Product_2004,Category_005,6000.0


In [22]:
# df = pd.read_csv('Data\\train.csv')
# df = df[(df['store']==1)]

# pd.to_datetime(df['date'].max()) - pd.to_datetime(df['date'].min())
# df = df.reset_index(drop=True)
# df.rename(columns={'date': 'Date', 'item':'Product_Code', 'sales':'Order_Demand'}, inplace=True)
# df = df[['Date', 'Product_Code', 'Order_Demand']]
# df['Date'] = pd.to_datetime(df['Date'])
# df['Product_Code'] = df['Product_Code'].astype('str')

In [23]:
print(df.info())
print('-------------------------')
print("")
print("The Number of unique")
print('-------------------------')
print('Product code:\t', df.Product_Code.nunique())
print('Category:\t', df.Product_Category.nunique())
print('-------------------------')
print("The Product Code:")
print("")
for i, code in enumerate(df['Product_Code'].unique()):
    print(i+1, code)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14464 entries, 0 to 14463
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              14464 non-null  datetime64[ns]
 1   Product_Code      14464 non-null  object        
 2   Product_Category  14464 non-null  object        
 3   Order_Demand      14464 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 452.1+ KB
None
-------------------------

The Number of unique
-------------------------
Product code:	 8
Category:	 5
-------------------------
The Product Code:

1 Product_0025
2 Product_0739
3 Product_0901
4 Product_1154
5 Product_1248
6 Product_1295
7 Product_1378
8 Product_2004


---

### Split the train and test set
- Input
     data: dataframe with dates and Demand data
     
- output
    - train:  2012-01-10 ~ 2015-12-26    (4.5 Years)
    - valid:  2015-12-26 ~ 2016-06-23    (0.5 Years)
    - test :  2016-06-24 ~ 2016-12-21   (0.5 Years)

In [24]:
# 훈련 데이터에서 IQR을 계산합니다.
def replace_outlier(product_df, train_end):

    train_df = product_df[:train_end]
    valid_test_df = product_df[train_end:]

    Q1 = train_df['y'].quantile(0.25)
    Q3 = train_df['y'].quantile(0.75)
    IQR = Q3 - Q1

    outlier_condition = (train_df['y'] < (Q1 - 1.5 * IQR)) | (train_df['y'] > (Q3 + 1.5 * IQR))
    mean_y = train_df.loc[~outlier_condition, 'y'].mean()

    train_df.loc[outlier_condition, 'y'] = mean_y
    noout_product_df = pd.concat([train_df, valid_test_df])
    
    return noout_product_df

In [25]:
# 훈련 데이터에서 IQR을 계산합니다.
def replace_outlier(product_df, train_end):


    Q1 = product_df['y'].quantile(0.25)
    Q3 = product_df['y'].quantile(0.75)
    IQR = Q3 - Q1

    outlier_condition = (product_df['y'] < (Q1 - 1.5 * IQR)) | (product_df['y'] > (Q3 + 1.5 * IQR))
    mean_y = product_df.loc[~outlier_condition, 'y'].mean()

    product_df.loc[outlier_condition, 'y'] = mean_y
    
    return product_df

In [26]:
def split_data(product_df, time_steps): 

#     train_end = len(product_df[product_df['Date']<'2015-12-26']) # train 데이터 개수
#     val_end = len(product_df[product_df['Date']<'2016-06-23']) # validation 데이터 개수
    val_end = len(product_df)-2*time_steps
    train_end = val_end - 2*time_steps
    y = product_df.filter(['y']).values # y(수요량) 값
    
    # Minmax로 0~1 사이에 값이 오도록 정규화
    sc = MinMaxScaler() # 객체 생성
    y_scaled = sc.fit_transform(y) # 전체 y값 정규화
    # Train Data
    #y_train_scaled = y_scaled[:train_end,:]
    X_train = [] 
    y_train = []
    for i in range(time_steps, train_end-time_steps): 
        X_train.append(y_scaled[i-time_steps:i,0]) 
        y_train.append(y_scaled[i:i+time_steps,0])

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    # Reshape X_train for LSTM -> (batch_size, time_steps, features)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
    y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1],1))
    
    # Validation Data
    X_val = []
    y_val = []
    
    X_val.append(y_scaled[train_end:train_end+time_steps, 0])
    y_val.append(y_scaled[train_end+time_steps:val_end, 0])

    X_val = np.array(X_val)
    y_val = np.array(y_val)
    # Reshape X_val for LSTM -> (batch_size, time_steps, features)
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1],1))
    y_val = np.reshape(y_val, (y_val.shape[0], y_val.shape[1],1))
    
    # Test Data
    X_test = []
    X_test.append(y_scaled[val_end:val_end+time_steps,0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
    
    y_test = product_df.iloc[val_end+time_steps:,:]
    y_test = y_test.copy()
    y_test['y_norm'] = y_scaled[val_end+time_steps:].reshape(-1).copy()
    # test data 개수만큼 반복
#     for i in range(val_end+time_steps, len(y_scaled)-time_steps):
#         X_test.append(y_scaled[i-time_steps : i, 0])

    return X_train, y_train, X_val, y_val, X_test, y_test, sc

## Optimized LSTM

In [27]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch
import tempfile

# 모델 구축 및 최적화 함수
def build_model(hp):
    
    model = Sequential()
    num_LSTM_layers = hp.Int('num_LSTM_layers', 1, 2)
    for i in range(num_LSTM_layers):
        if i == 0:
            model.add(LSTM(units=hp.Int(f'LSTM_units_{i+1}', min_value=64, max_value=256, step=32),
                           activation= 'tanh',
                           input_shape=(None,1),
                           return_sequences=True))
        else:
            model.add(LSTM(units=hp.Int(f'LSTM_units_{i+1}', min_value=64, max_value=256, step=32),
                           activation= 'tanh',
                           return_sequences=True))
    
    #model.add(Dropout(0.2))
    num_Dense_layers = hp.Int('num_Dense_layers', 1, 2)
    for i in range(num_Dense_layers):
        if i == 0:
            model.add(TimeDistributed(Dense(hp.Int(f'Dense_units_{i+1}', min_value=16, max_value=64, step=16), 
                                        activation= 'relu')))
        else:
            model.add(TimeDistributed(Dense(hp.Int(f'Dense_units_{i+1}', min_value=16, max_value=32, step=16), 
                                        activation= 'relu')))
        
    model.add(TimeDistributed(Dense(1)))

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 5e-4])),
                  loss='mean_squared_error',
                  metrics='mae')

    return model

# 모델 학습 및 평가 함수
def optimize_model(X_train, y_train, X_val, y_val, X_test, sc, epochs, trials):
    with tempfile.TemporaryDirectory() as temp_dir:
        tuner = RandomSearch(
            build_model,
            objective='val_loss',
            max_trials= trials,
            directory=temp_dir,
            project_name='temp_project')

    tuner.search_space_summary()

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    tuner.search(X_train, y_train,
                 epochs=epochs,
                 batch_size=32,
                 validation_data=(X_val, y_val),
                 callbacks=[early_stopping])

    tuner.results_summary()

    best_model = tuner.get_best_models(num_models=1)[0]

    # Test 데이터 예측
    pred = best_model.predict(X_test) # 예측값 얻기
    pred_norm = pred # 예측값을 저장하되, normalize된 값 저장
    pred = sc.inverse_transform(pred.reshape(pred.shape[0], pred.shape[1])) # denormalize된 예측값 저장
    
    best_model.summary()
    # 모델 객체와 예측값 반환
    return best_model, pred, pred_norm

In [28]:
 def LSTM_single(product_df, time_steps, epochs, trials):

    # 학습 데이터와 테스트 데이터 분리
    X_train, y_train, X_val, y_val, X_test, y_test, sc = split_data(product_df, time_steps)
    
    # LSTM 모델 학습 및 예측
    best_model, pred, pred_norm = optimize_model(X_train, y_train, X_val, y_val, X_test, sc, epochs, trials)
    
    # 예측 결과 저장
    y_test.reset_index(drop=True, inplace=True)
    pred_df = pd.DataFrame({'Pred': pred.reshape(-1) ,'Pred_norm': pred_norm.reshape(-1)})
    res_df = pd.concat([y_test, pred_df], axis=1)
    res_df.set_index('Date', inplace=True)
    res_df.loc[res_df['Pred']<0, 'Pred']=0
    
    # res_df: ['y', 'y_norm', 'Pred', 'Pred_norm'], index='Date'
    # 모델과 result_df
    return best_model, res_df

## Plot the result

In [29]:
def actual_pred_plot(product_code, res_df, metric_df, normalize):
    today = date.today()
    """
    Plot the actual vs predition and save the figure in the given directory
    """
    
    save_path = os.path.join("Result", "Single_LSTM_Result", product_code)
    save_name = f'{product_code}_all_result'
    
    title = f"Pred Actual Plot - {product_code}"
    actual = res_df['y']
    pred = res_df['Pred']
    
    if normalize: 
        title += "(Normalized)"
        actual = res_df['y_norm']
        pred = res_df['Pred_norm']
        save_name += "_normalized"
    # Plot   
    plt.figure(figsize=(16, 8))
    plt.title(title, fontsize=20)
    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Order Demand", fontsize=14)
    plt.plot(actual, label ='Actual', alpha=0.6, marker='o', ms=3)
    plt.plot(pred, label='Prediction', alpha=0.8, marker='o', ms=3)
    plt.legend(loc="upper right")
        
    # Plot 결과 저장
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    # save the figure
    today_date = f'_{today.month:02d}{today.day:02d}'
    plt.savefig(os.path.join(save_path, save_name+'.png'))
    # Metric도 함께 저장
    metric_df.to_csv(os.path.join(save_path, save_name+'.csv'))
        
    plt.close('all') # close all figures to free up memory

## Save and Load the model 

In [30]:
def save_model(product_code, best_model):
    today = date.today()
    folder_path = 'Result/Single_LSTM_Result/Model'
    file_name = f'{product_code}_{today.month:02d}{today.day:02d}.pkl'
    save_path = os.path.join(folder_path, file_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    # 객체를 pickle 파일로 저장
    with open(save_path, 'wb') as f:
        pickle.dump(best_model, f)
    return best_model

In [31]:
def load_model(file_name):
    file_path = f'Result/Single_LSTM_Result/Model/{file_name}'
    
    with open(file_path, 'rb') as file:
        model_dict= pickle.load(file)
    
    return best_model

## Metrics

In [32]:
# Model Metric
def mase(training_series, testing_series, prediction_series):
    n = training_series.shape[0]
    d = np.abs(np.diff(training_series)).sum() / (n-1)
    
    errors = np.abs(testing_series - prediction_series)
    return errors.mean() / d

def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual+1)))

# 정규화 된 지표
def nrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, squared=False)
    target_mean = np.mean(y_true)
    nrmse = mse / target_mean
    return nrmse

# 정규화 된 지표
def nmae(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    target_mean = np.mean(y_true)
    nmae = mae / target_mean
    return nmae

In [33]:
def calculate_metrics(product_code, res_df, normalize):
    # 정규화 옵션이 True인 경우 정규화된 데이터 사용, 그렇지 않으면 원래 데이터 사용
    if normalize:
        actual = res_df['y_norm']
        pred = res_df['Pred_norm']
    else:
        actual = res_df['y']
        pred = res_df['Pred']

    # 메트릭 계산
    # MASE = mase(np.array(train_series), np.array(actual), pred) 
    MAPE = mape(actual, pred) 
    RMSE = mean_squared_error(actual, pred)**0.5 
    MAE = mean_absolute_error(actual,pred) 
    NRMSE = nrmse(actual,pred) 
    NMAE = nmae(actual,pred)
    R2 = r2_score(actual,pred) 
    # RMSLE = mean_squared_log_error(actual, pred)**0.5 

    # 계산된 메트릭을 데이터프레임에 추가
    metric_df = pd.DataFrame({'MAPE':[round(MAPE, 4)],
                           'RMSE':[round(RMSE, 4)],
                           'MAE':[round(MAE, 4)],
                           'NRMSE':[round(NRMSE, 4)],
                           'NMAE':[round(NMAE, 4)],
                           'R2':[round(R2, 4)]},
                            index= [product_code])

    return metric_df

---

## Check the Result

## Whole Process
    - product_code에 str으로 예측하고자 하는 코드를 입력
    - ['Product_0025', 'Product_0739', 'Product_0901', 'Product_1154',
       'Product_1248', 'Product_1295', 'Product_1378', 'Product_2004']

In [20]:
def execute_single_LSTM(product_code, time_steps=90, epochs=100, optimize_trials=10):
    start_time = time.time()
    product_code = product_code # 예측하고자 하는 코드 입력
    product_df = df[df['Product_Code']== product_code].reset_index(drop=True)
    product_df = product_df[['Date', 'Order_Demand']]
    product_df.rename(columns={'Order_Demand': 'y'}, inplace=True)

    # LSTM 단일 모델
    model, res_df = LSTM_single(product_df, time_steps, epochs, optimize_trials) #dictionary, time_steps, epochs
    save_model(product_code, model)
    # 모델 Metric과 Pred_Actual Plot 저장
    metric_df_norm = calculate_metrics(product_code, res_df, True)
    metric_df= calculate_metrics(product_code, res_df, False)
    
    actual_pred_plot(product_code, res_df, metric_df_norm, True)
    actual_pred_plot(product_code, res_df, metric_df, False)
    # 실행시간 확인
    elapsed_time_seconds = time.time() - start_time
    elapsed_time_minutes = elapsed_time_seconds / 60
    print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))
    return metric_df

---

In [21]:
for code in ['Product_0025', 'Product_0739', 'Product_0901', 'Product_1154',
             'Product_1248', 'Product_1295', 'Product_1378', 'Product_2004']:
    print("==================================")
    print(f"========== { code } ==========")
    print("==================================")
    execute_single_LSTM(code)

Trial 10 Complete [00h 00m 08s]
val_loss: 0.012715483084321022

Best val_loss So Far: 0.011502848006784916
Total elapsed time: 00h 03m 01s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in C:\Users\7info\AppData\Local\Temp\tmpp0s0qhd3\temp_project
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 03 summary
Hyperparameters:
num_LSTM_layers: 1
LSTM_units_1: 256
num_Dense_layers: 1
Dense_units_1: 48
learning_rate: 0.01
LSTM_units_2: 64
Dense_units_2: 16
Score: 0.011502848006784916

Trial 06 summary
Hyperparameters:
num_LSTM_layers: 1
LSTM_units_1: 128
num_Dense_layers: 1
Dense_units_1: 16
learning_rate: 0.0005
LSTM_units_2: 64
Dense_units_2: 32
Score: 0.012617578729987144

Trial 00 summary
Hyperparameters:
num_LSTM_layers: 2
LSTM_units_1: 128
num_Dense_layers: 1
Dense_units_1: 16
learning_rate: 0.0005
LSTM_units_2: 64
Score: 0.012673303484916687

Trial 07 summary
Hyperparameters:
num_LSTM_layers: 1
LSTM_units_1: 256
num_Dense_layers: 2
Dense_units_1